In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
glove_wiki_vectors.most_similar("myanmar")

[('burma', 0.8043322563171387),
 ('cambodia', 0.7673672437667847),
 ('laos', 0.7371729612350464),
 ('thailand', 0.7355332374572754),
 ('nepal', 0.7052156925201416),
 ('junta', 0.6958188414573669),
 ('malaysia', 0.678316593170166),
 ('bangladesh', 0.6724640130996704),
 ('vietnam', 0.6715803742408752),
 ('sudan', 0.6704097390174866)]

In [7]:
glove_wiki_vectors.most_similar("chicken")

[('meat', 0.818773090839386),
 ('pork', 0.7934611439704895),
 ('cooked', 0.7839199304580688),
 ('roast', 0.7816914319992065),
 ('soup', 0.7801428437232971),
 ('fried', 0.7603550553321838),
 ('beef', 0.7500470280647278),
 ('shrimp', 0.7424989342689514),
 ('sausage', 0.7347979545593262),
 ('lamb', 0.7248150706291199)]

In [8]:
glove_wiki_vectors.most_similar("general")

[('secretary', 0.7606937289237976),
 ('chief', 0.7243471145629883),
 ('gen.', 0.689914882183075),
 ('president', 0.6798181533813477),
 ('vice', 0.6726594567298889),
 ('office', 0.6624852418899536),
 ('deputy', 0.6548395752906799),
 ('department', 0.6539177298545837),
 ('commander', 0.6525410413742065),
 ('led', 0.6484827995300293)]

In [9]:
glove_wiki_vectors.most_similar("aung")

[('kyi', 0.9241669774055481),
 ('suu', 0.9210778474807739),
 ('shwe', 0.707683265209198),
 ('daw', 0.6803808212280273),
 ('nld', 0.6803585886955261),
 ('maung', 0.6355515718460083),
 ('thein', 0.6310358047485352),
 ('khin', 0.6292538046836853),
 ('junta', 0.6089261770248413),
 ('myint', 0.5971934199333191)]

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])

print(coast_shore_similarity)
print(clothes_closet_similarity)
print(old_new_similarity)
print(smart_intelligent_similarity)
print(dog_cat_similarity)
print(tree_lawyer_similarity)

0.70002717
0.54627603
0.6432488
0.7552733
0.8798075
0.076719455


In [13]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [14]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [15]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [16]:
glove_wiki_vectors.similarity("white", "rich")

0.44723597

In [17]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [18]:
analogy("male", "sciences", "female")

# I do not think there are any biases here.

male : sciences :: female : ?


,Analogy word,Score
0,science,0.798229
1,institute,0.772056
2,humanities,0.753807
3,engineering,0.737927
4,academy,0.692104
5,mathematics,0.680356
6,arts,0.677597
7,physics,0.674361
8,chemistry,0.668080
9,biomedical,0.665550


In [19]:
analogy("male", "father", "female")

# I do not think there are any biases here.

male : father :: female : ?


,Analogy word,Score
0,brother,0.876142
1,son,0.854832
2,mother,0.845568
3,wife,0.838224
4,friend,0.832187
5,grandfather,0.828927
6,husband,0.818180
7,uncle,0.813991
8,daughter,0.803184
9,cousin,0.770455


In [20]:
analogy("male", "engineer", "female")

# I do not think there are any biases here.

male : engineer :: female : ?


,Analogy word,Score
0,engineers,0.679807
1,technician,0.665169
2,contractor,0.646667
3,officer,0.640510
4,worked,0.637603
5,architect,0.636746
6,engineering,0.632931
7,mechanic,0.632612
8,retired,0.621830
9,chemist,0.594601


In [21]:
analogy("male", "child", "female")

# I think it's fair and unbiased here too. 

male : child :: female : ?


,Analogy word,Score
0,children,0.811977
1,woman,0.778451
2,mother,0.767360
3,girl,0.750344
4,parents,0.734957
5,pregnant,0.729554
6,boy,0.719443
7,daughter,0.703899
8,wife,0.698309
9,young,0.698086


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

1. I think the quality is great. For instances such as finding the similarities between ("dog", "cat") which is high and ("tree", "lawyer") which is low. For analogy("male", "engineer", "female"), the highest is engineers or technician which is expected and unbiased. 

2. The use of the word embeddings that contain biased associations can perpetuate gender biases and stereotypes, and harm in educational settings for children who may unknowingly absorb and reinforce such biases. Moreover, biased embeddings in NLP tasks, such as sentiment analysis and automated hiring processes, can result in biased decisions or recommendations, further perpetuating inequalities and discriminatory practices.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [22]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [23]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [24]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [25]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [26]:
from sklearn.metrics import classification_report

pipe = make_pipeline(
    CountVectorizer(stop_words="english"),
    LogisticRegression(max_iter=1000)
)
pipe.fit(X_train, y_train)

bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)
bow_report = classification_report(y_test, pipe.predict(X_test), digits=2) # use digits=2

print(bow_train_accuracy)
print(bow_test_accuracy)


0.9562051178314959
0.8173666823973572


In [27]:
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

# Adapted from lecture 17

In [28]:
X_train_embeddings

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.690874,-0.755216,-3.279650,-1.111956,2.333812,1.712701,0.346985,5.052109,-1.078272,1.564646,...,0.978959,0.228331,0.843013,-1.340969,-0.884684,1.971317,-0.880458,-0.599687,-5.299727,-0.139515
1,0.021659,-1.051312,-4.063361,-1.818072,3.027572,-1.226410,0.035204,2.806751,-0.181556,1.530708,...,0.873182,-1.440154,0.219029,-1.426304,-1.591761,-0.475234,-0.122053,0.695197,-4.421231,-0.027261
2,-0.101639,-1.674547,-1.391433,-3.230565,0.636887,0.756545,0.205278,5.351284,-2.872737,3.033967,...,0.594443,0.435623,-0.556652,-2.759545,-1.521208,0.728254,-0.365033,1.230975,-5.989079,0.142662
3,-1.502830,-0.245505,-2.919757,-1.312146,3.315014,-0.281209,1.019417,3.909962,-0.440415,0.797455,...,0.452126,-0.261931,-0.375499,-0.218783,-1.355882,0.384430,0.619531,0.548577,-3.217188,0.119259
4,-0.503838,-0.450076,-4.626111,-2.820389,4.793096,-0.805586,1.518527,5.712772,1.548753,1.589522,...,-0.293538,-0.217686,-3.700689,-1.188137,0.648456,-2.548641,-0.413536,1.096762,-3.927443,-1.008204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9882,-0.243877,0.100877,-5.652421,-2.358680,4.555353,0.425443,0.512634,3.707028,0.320085,0.068646,...,0.637390,-2.082109,-2.228711,-1.563666,-2.143045,-1.513141,0.384336,2.224789,-7.159804,-2.029576
9883,-1.739380,-0.460984,-2.697332,-2.143512,2.983281,-0.489825,0.852471,4.899079,-1.010702,1.982431,...,1.204611,-0.272654,-0.468054,-0.990877,-1.541753,1.042608,0.475310,-0.484631,-4.092132,1.285569
9884,-1.059491,2.290304,-3.937229,-1.183871,-1.614022,0.741002,1.600839,4.473727,-3.815495,2.880342,...,3.602502,-0.177909,-0.488643,-1.663591,-1.408746,-0.145548,1.216508,1.783865,-3.802025,2.775653
9885,-2.270314,-2.764440,-2.608192,-2.656994,-1.151195,1.417525,1.212543,3.261775,-3.201015,3.732340,...,1.658000,0.443025,2.027175,-2.207927,-3.366125,1.169303,2.571275,2.802700,-3.126972,-0.493300


In [29]:
X_test_embeddings

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.958416,3.125293,-4.034988,-2.041907,2.595106,0.445679,0.386465,5.083802,-1.852383,1.560549,...,-0.379911,-0.726037,-1.127668,-0.563859,0.177253,-0.294613,-0.855359,1.358393,-7.270348,1.399086
1,-1.299894,0.630366,-2.318005,-2.507104,4.050345,-0.310809,0.272773,3.979269,1.976808,0.731067,...,0.931720,0.025334,-0.485929,-0.743087,-1.798334,2.132924,0.299637,0.670521,-3.445617,0.822201
2,1.671838,-3.753560,-2.243932,-6.034901,-2.993384,2.103476,-0.182704,1.314220,0.393286,0.491056,...,-0.021817,0.214875,-1.920058,-0.585280,-2.042112,1.066200,-0.503640,0.496954,-0.952958,-0.096886
3,3.285316,-1.347252,-5.520160,-1.793326,-0.319224,-4.395940,-0.787140,3.501826,-2.966559,4.460822,...,2.226200,-1.617096,1.458074,-4.914721,-2.545942,-3.723066,-2.130358,1.875884,-8.811860,-0.704100
4,-3.296173,1.518396,-3.726718,-1.118125,1.690331,-1.578362,0.601601,5.086967,-0.884238,3.076164,...,0.435902,0.211110,1.393579,0.039486,-2.776130,1.652559,3.089273,2.418594,-3.963638,0.478087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1.557899,3.393842,-4.641134,-2.090698,-0.534571,1.281198,0.293231,2.920421,-4.139679,3.409355,...,1.212196,-0.208200,0.488157,-2.472521,-0.779388,-2.337183,-1.017759,2.821771,-6.786845,1.004315
4234,0.898568,0.069064,-4.769829,-1.440225,0.809664,-0.419533,0.581546,5.336606,-2.643578,3.025955,...,0.757724,-0.297941,1.082521,-2.018424,-1.251820,-1.624845,1.181356,0.403363,-5.941779,0.078516
4235,-1.105252,0.138040,-3.517998,-1.480571,3.263637,0.550350,-0.424693,5.483697,-0.373895,0.968357,...,1.648454,-0.816772,-0.055126,-1.346399,-2.789468,0.928694,1.086834,0.395372,-4.317700,0.184079
4236,0.427034,0.924937,-3.550213,-3.172181,0.836191,-0.167859,1.038291,4.872327,-0.155426,3.150545,...,1.685842,-1.522991,1.195588,-2.083883,-1.750561,1.283785,-0.016592,1.414247,-4.331584,-0.024904


In [30]:
lr = LogisticRegression(max_iter=2000)
lr.fit(X_train_embeddings, y_train)
emb_train_accuracy = lr.score(X_train_embeddings, y_train)
emb_test_accuracy = lr.score(X_test_embeddings, y_test)
emb_report = classification_report(y_test, lr.predict(X_test_embeddings), digits=2) # use digits=2

print(emb_train_accuracy)
print(emb_test_accuracy)


0.8493981996561141
0.7928268050967437


/Users/sittpaing/miniconda3/envs/cpsc330/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
grader.check("q1.5")

q1.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

1. CountVectorizer counts the frequency of words in the text and creates a sparse matrix where each row represents a document, and each column represents a unique word. The occurrence of words in the text is captured. The average embedding approach involves computing the average vector representation of words or tokens in the text. By averaging the word embeddings, fixed-length vector representation for the entire text is captured.

2. CountVectorizer is more interpretable because it directly represents the occurrence of words, while the average embedding is less interpretable since it relies on dense vector embeddings learned from pre-trained models. Therefore, the relationship between individual words and the resulting vector is less transparent.

3. : The average embedding approach uses pre-trained word embeddings that are trained on different styles of text so transfer learning is used here. We benefit from the knowledge acquired during their training on totally different corpus even when the training data is limited. The training data just fine tunes the embeddings so the accuracy is quite high with little training. In this case, it is not overfitting the train data yet the accuracy is high. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [32]:
from sklearn.datasets import fetch_20newsgroups

In [33]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [34]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [35]:
# Adapted from lecture 17
import re

def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
            and not token.like_email  # Check if it's an email address
            and not re.match(r"(http|https)://[^\s]*", token.lower_)  # Check if it's a URL
            and len(token.text) > 1  # Exclude words with one letter
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [36]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [37]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 ----------...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure need correct...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

Stopword removal: Tokens that are identified as stopwords are filtered out. Stopwords are not relevant or informative in this case.

Getting rid of URLs and email addresses: URLs and email addresses are not informative for topic modeling tasks. Removing them helps to focus on the more relevant content of the text.

Minimum token length: Tokens with a length less than min_token_len are filtered out. This step removes short and not very informative tokens. This ensures that only meaningful words are retained.

Irrelevant POS tags: Tokens with POS tags specified in the irrelevant_pos list are removed. Removing these tags helps eliminate noise and focus on meaningful words.

Lemmatization: Lemmatization reduces words to their base or dictionary form and this normalization step aids in capturing the underlying meaning of words.

Lowercasing: The lemmatized tokens are converted to lowercase to ensure that words with different letter cases are treated as the same word.

These preprocessing steps clean and transform the raw text into a more standardized and meaningful representation for topic modeling by removing irrelevant elements and enhance the quality and effectiveness of topic modeling.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [38]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", random_state=0
)


I choose 8 as the number of topics as the target has 8 categories and it's reasonable to think of each category as a topic.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [39]:
vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

In [40]:
document_topics = lda.fit_transform(X)
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
god           people        people        gun           window        
think         know          say           weapon        widget        
good          question      armenian      firearm       use           
know          think         know          law           server        
time          entry         israel        crime         application   
jesus         believe       armenians     state         display       
people        god           come          control       run           
come          exist         turkish       people        set           
say           use           kill          year          problem       
year          point         jews          rate          motif         


topic 5       topic 6       topic 7       
--------      --------      --------      
game          libxmu        file          
team          den

In [41]:
topic_labels = [ 
    "soc.religion.christian", # topic 0
    "alt.atheism", # topic 1
    "talk.politics.mideast", # topic 2
    "talk.politics.guns", # topic 3
    "comp.windows.x", # topic 4
    "rec.sport.hockey",# topic 5
    "rec.sport.baseball", # topic 6
    "comp.graphics", # topic 7 
]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [42]:
pd.DataFrame((document_topics*100).round(), index=df['text_pp'], columns=[t + '%' for t in topic_labels]).head()

,soc.religion.christian%,alt.atheism%,talk.politics.mideast%,talk.politics.guns%,comp.windows.x%,rec.sport.hockey%,rec.sport.baseball%,comp.graphics%
text_pp,,,,,,,,
know read u.s.c. 922 sence wonder help u.s.c. 922 provide paragraph shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute 922 enforce individual tell government tell possess pass law law know law constitional go court go court run mill guilty jail claim right possess tell supreme court right,0.0,63.0,14.0,21.0,0.0,0.0,0.0,0.0
bad question ref list algorithm think bit hard point find plane define circle lie algorithm find center circle line pass center perpendicular plane point pass center sphere repeat unused point original point give different line pass sphere origin interection center sphere radius easy compute distance center original point leave math workable algorithm alternate method pair point plane form perpendicular bisector line segment pair contain center sphere pair form plane intersect point easy implement,0.0,99.0,0.0,0.0,0.0,0.0,0.0,0.0
actuallay hand support idea have newsgroup aspect graphic programming propose brian reply original posting suggest loose structure comp.graphics.programmer reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news forum exchange idea help help think difficult achive different thing meeting good evening netters|-,23.0,43.0,0.0,0.0,6.0,0.0,0.0,27.0
follow problem bug appreciate help create window child root event_mask buttonpressmask|keypressmask child do_not_propagate_mask buttonpressmask|keypressmask keypress event discard buttonpress event fall subwindow set fyi xnew olvwm wrong,0.0,21.0,0.0,0.0,72.0,0.0,5.0,0.0
late upi foreign ministry spokesman ferhat ataman tell journalist turkey close air space flight armenia prevent humanitarian aid reach republic overland turkish territory uncivilized people exhibit sign compassion allow humanitarian aid reach civilian population nazis turkey pronounce hypocrite choose continue condemnation serbians,0.0,0.0,80.0,0.0,19.0,0.0,0.0,0.0


The topic assignment for doc 1 is atheism with 63%.
The topic assignment for doc 2 is atheism with 99%.
The topic assignment for doc 3 is atheism with 43%, christian with 23% and graphics with 27%.
The topic assignment for doc 4 is windows.x with 72%.
The topic assignment for doc 5 is mideast with 80%.

From this 5 documents, it is shown that graphics is not correctly predicted but upon looking at multiple documents after these 5, this preprocessing seems to predict many of them correctly. Other preprocessing makes it worse.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

Adapted from lecture 18 and sklearn.

Decision Trees are capable of handling multiclass classification tasks. A common technique is to reduce multiclass classification into several instances of binary classification problems such as the one-vs-rest approach or
the one-vs-one approach.

Knn can handle multiclass classification since it classifies a new sample based on the majority class of its K nearest neighbors. Scikit-learn's KNeighborsClassifier can be used for multiclass classification with KNN.

Similar to decision tree, random forest can handle multiclass classification tasks by using the one-vs-rest or one-vs-one strategy.

Similarly, multiclass classification can be handled with one-vs-rest where the class with the highest score "wins" and that's going to be the prediction for this class, or using multinomial logistic regression.

SVM RBF can handle multiclass according to a one-vs-one scheme. In total, n_classes * (n_classes - 1) / 2 classifiers are constructed and each one trains data from two classes. It also allows to monotonically transform the results of the “one-versus-one” classifiers to a “one-vs-rest” decision function.

2. Transfer learning in natural language processing (NLP) or computer vision refers to the technique of leveraging pre-trained models on a large dataset and transferring their learned knowledge to new, related tasks or domains. It fine tunes the knowledge gained from pre-training to improve the performance on a specific task with limited training data.

3. Pre-trained models, such as those trained on large datasets like ImageNet have already learned valuable feature representations from the input data. By replacing or fine-tuning only some of the layers and adapting them to a specific task, the neural net can leverage the learned representations and adapt them to the new task and if anyone wants to train an entire CNN from scratch, it requires a large dataset, powerful computers, and a huge amount of human effort to train the model.






<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)